In [ ]:
import itertools
import time
import params
from QAOA import QAOArunner
from datetime import date
import pandas as pd
from solver import Solver
import ray
import numpy as np
import yagmail
import logging

from MaxCutProblem import MaxCutProblem
problem = MaxCutProblem()

with open("email_credentials.txt", "r") as f:
    email_password = f.read().strip()
import ast
import networkx as nx

settings = [('uniform', 'COBYLA', 'multiangle', False, True, 3), ('gaussian', 'COBYLA', 'multiangle', False, True, 3)]

logging.basicConfig(level=logging.INFO)

@ray.remote
def parallell_runner(parameters, graph,name):
    timestamp = time.time()
    date_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp))
    print(f"Processing task {parameters}, {name} at time {date_time}")
    qaoa = QAOArunner(graph, simulation=True, param_initialization=parameters[0], optimizer =  parameters[1], 
    qaoa_variant=parameters[2], warm_start=parameters[3], errors = parameters[4],depth = parameters[5])
    qaoa.build_circuit()
    qaoa.run()
    solver = Solver(graph)
    bitstring, value = solver.solve()
    end_time  = time.time()
    date_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time))
    print(f"Solved task {parameters}, {name} at time {date_time}. It took {end_time-timestamp} seconds.")
    return {'param_initialization': parameters[0], 'optimizer': parameters[1],'qaoa_variant': parameters[2], 'warm_start' : parameters[3], 
    'errors':parameters[4], 'depth' : parameters[5], 'graph_size': len(graph.nodes()), 'graph_name' : name,
        'time_elapsed': qaoa.time_elapsed, 'quantum_func_evals': qaoa.fev, 'obj_func_evolution': qaoa.objective_func_vals,
        'quantum_solution':qaoa.solution, 'quantum_obj_value' : qaoa.objective_value, 
        'classic_solution' : bitstring, 'classic_value': value }


if ray.is_initialized():
    ray.shutdown()
    print('Shutting down old Ray instance.')
ray.init(num_cpus=15)

data = []

graphs, names = [],[]


for i in range(9,4, -1):
    graphs_i, names_i = problem.get_test_graphs(i)
    graphs.append(graphs_i) #TODO: check that this works for very small values
    names.append(names_i)
print(len(graphs))
graphs = list(itertools.chain.from_iterable(graphs))
names = list(itertools.chain.from_iterable(names))

combos = [settings, graphs]
print('combos', len(combos))

all_combos = list(itertools.product(*combos))


all_combos = [combo + (names[graphs.index(combo[1])],) for combo in all_combos]
all_combos_dict = [{"parameters": combo[0], "graph": combo[1], "name": combo[2]} for combo in all_combos]
#TODO: make dictionary
print('len all_combos',len(all_combos))

print('Settings:', settings)

futures = [parallell_runner.remote(parameters, graph, name) for parameters, graph, name in all_combos]

result_ids, unfinished = ray.wait(futures, timeout = 60*60*20, num_returns = len(all_combos))
for task in unfinished:
    ray.cancel(task)
underway_df = pd.DataFrame(ray.get(result_ids))
underway_df.to_csv(f'results_underway.csv', mode='a', header=False)
data.extend(ray.get(result_ids))
print(f'Done with Parameters: {settings} at time: {time.time()}')


df = pd.DataFrame(data)
df.to_csv('data.csv', index=False)

yag = yagmail.SMTP("torbjorn.solstorm@gmail.com", email_password)
recipient = "torbjorn.smed@gmail.com"
subject = "Data from Python Script"
body = "Here is the data you requested."
attachment = "data.csv"

yag.send( subject=subject, contents=body, attachments=attachment)
print("Email sent successfully!")


df = pd.DataFrame(data)
df.to_csv(f'results_.csv')


2024-12-09 13:15:20,877	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-09 13:15:24,056	INFO worker.py:1810 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


5
combos 2
len all_combos 200
Settings: [('uniform', 'COBYLA', 'multiangle', False, True, 3), ('gaussian', 'COBYLA', 'multiangle', False, True, 3)]
(parallell_runner pid=6492) Processing task ('uniform', 'COBYLA', 'multiangle', False, True, 3), H?r@fR] at time 2024-12-09 13:15:33
(parallell_runner pid=1760) Processing task ('uniform', 'COBYLA', 'multiangle', False, True, 3), H?qlvNx at time 2024-12-09 13:15:34
(parallell_runner pid=30816) Processing task ('uniform', 'COBYLA', 'multiangle', False, True, 3), HCQbQpJ at time 2024-12-09 13:15:34
(parallell_runner pid=13660) Processing task ('uniform', 'COBYLA', 'multiangle', False, True, 3), H?bFUi~ at time 2024-12-09 13:15:35
(parallell_runner pid=31444) Processing task ('uniform', 'COBYLA', 'multiangle', False, True, 3), H?`vFVt at time 2024-12-09 13:15:35
(parallell_runner pid=32260) Processing task ('uniform', 'COBYLA', 'multiangle', False, True, 3), HCQeLr~ at time 2024-12-09 13:15:35
(parallell_runner pid=27196) Processing task ('uni